#Notebook for testing performance of Visaul Recognition Custom Classifiers
[Watson Developer Cloud](https://www.ibm.com/watsondevelopercloud) is a platform of cognitive services that leverage machine learning techniques to help partners and clients solve a variety of business problems. Furthermore, several of the WDC services fall under the **supervised learning** suite of machine learning algorithms, that is, algorithms that learn by example. This begs the questions: "How many examples should we provide?" and "When is my solution ready for prime time?"

It is critical to understand that training a machine learning solution is an iterative process where it is important to continually improve the solution by providing new examples and measuring the performance of the trained solution. In this notebook, we show how you can compute important Machine Learning metrics (accuracy, precision, recall, confusion_matrix) to judge the performance of your solution. For more details on these various metrics, please consult the **[Is Your Chatbot Ready for Prime-Time?](https://developer.ibm.com/dwblog/2016/chatbot-cognitive-performance-metrics-accuracy-precision-recall-confusion-matrix/)** blog.


<br> The notebook assumes you have already created a Watson [Visual Recognition](https://www.ibm.com/watson/developercloud/visual-recognition.html) instance and trained [custom classifiers](https://www.ibm.com/watson/developercloud/doc/visual-recognition/tutorial-custom-classifier.html). </br>
<br> To leverage this notebook, you need to provide the following information</br>
* Credentials for your Visual Recognition instance (apikey)
* id for your trained classifier (this is returned when you train your Visual Recognition custom classifier)
* csv file with your test images (paths to images on your local disk) and corresponding class labels
* results csv file to write the results to (true vs. predicted class labels)
* csv file to write confusion matrix to

Note that the input test csv file should have a header with the fields **image** and **class**. 

In [8]:
#Import utilities
import json
import csv
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import pandas_ml
from pandas_ml import ConfusionMatrix
from watson_developer_cloud import VisualRecognitionV3

Provide the path to the parms file which includes credentials to access your VR service as well as the input
test csv file and the output csv files to write the output results to.

In [ ]:
# Provide complete path to the file which includes all required parms
# A sample parms file is included (example_VR_parms.json)
vrParmsFile = 'PATH to your Visual Recognition PARMS file'
parms = ''
with open(vrParmsFile) as parmFile:
    parms = json.load(parmFile)

url=parms['url']
apikey=parms['apikey']
vr_id=parms['vr_id']
test_csv_file=parms['test_csv_file']
results_csv_file=parms['results_csv_file']
confmatrix_csv_file=parms['confmatrix_csv_file']

json.dumps(parms)

# Create an object for your Visual Recognition instance
visual_recognition = VisualRecognitionV3('2016-05-20', api_key=apikey)

Define useful methods to classify using custom VR classifier.

In [17]:
# Given an image and a pointer to VR instance and classifierID, get back VR response
def getVRresponse(vr_instance,classifierID,image_path):
    with open(image_path, 'rb') as image_file:
        image_results = vr_instance.classify(images_file=image_file,
                                              threshold=0.1,
                                              classifier_ids=[classifierID])
        #print(json.dumps(image_results, indent=2))
    return image_results

# Process multiple images (provided via csv file) in batch. Effectively, read the csv file and for each image
# utterance, get VR response. Aggregate and return results.
def batchVR(vr_instance,classifierID,csvfile):
    test_classes=[]
    vr_predict_classes=[]
    vr_predict_confidence=[]
    images=[]
    i=0
    with open(csvfile, 'rb') as csvfile:
        csvReader=csv.DictReader(csvfile)
        for row in csvReader:
            test_classes.append(row['class'])
            vr_response = getVRresponse(vr_instance,classifierID,row['image'])
            vr_predict_classes.append(vr_response['images'][0]['classifiers'][0]['classes'][0]['class'])
            vr_predict_confidence.append(vr_response['images'][0]['classifiers'][0]['classes'][0]['score'])
            images.append(row['image'])
            i = i+1
            if(i%250 == 0):
                print "Processed ", i, " records"
        print "Finished processing ", i, " records"        
    return test_classes, vr_predict_classes, vr_predict_confidence, images

# Plot confusion matrix as an image
def plot_conf_matrix(conf_matrix):
    plt.figure()
    plt.imshow(conf_matrix)
    plt.show()

# Print confusion matrix to a csv file
def confmatrix2csv(conf_matrix,labels,csvfile):
    with open(csvfile, 'wb') as csvfile:
        csvWriter = csv.writer(csvfile)
        row=list(labels)
        row.insert(0,"")
        csvWriter.writerow(row)
        for i in range(conf_matrix.shape[0]):
            row=list(conf_matrix[i])
            row.insert(0,labels[i])
            csvWriter.writerow(row)
            

In [ ]:
# This is an optional step to quickly test response from Visual Recognition for a given image
#testImage='PATH to a test image'
#results = getVRresponse(visual_recognition,vr_id,testImage)
#print(json.dumps(results, indent=2))

Call Visual Recognition on the specified csv file and collect results.

In [ ]:
test_classes,vr_predict_classes,vr_predict_conf,images=batchVR(visual_recognition,vr_id,test_csv_file)

In [22]:
# print results to csv file including original text, the correct label, 
# the predicted label and the confidence reported by NLC.
csvfileOut=results_csv_file
with open(csvfileOut, 'wb') as csvOut:
    outrow=['image','true class','VR Predicted class','Confidence']
    csvWriter = csv.writer(csvOut,dialect='excel')
    csvWriter.writerow(outrow)
    for i in range(len(images)):
        outrow=[images[i],test_classes[i],vr_predict_classes[i],str(vr_predict_conf[i])]
        csvWriter.writerow(outrow)

In [23]:
# Compute confusion matrix
labels=list(set(test_classes))
vr_confusion_matrix = confusion_matrix(test_classes, vr_predict_classes, labels)
vrConfMatrix = ConfusionMatrix(test_classes, vr_predict_classes)

In [24]:
# Print out confusion matrix with labels to csv file
confmatrix2csv(vr_confusion_matrix,labels,confmatrix_csv_file)

In [ ]:
%matplotlib inline
vrConfMatrix.plot()

In [ ]:
# Compute accuracy of classification
acc=accuracy_score(test_classes, vr_predict_classes)
print 'Classification Accuracy: ', acc

In [ ]:
# print precision, recall and f1-scores for the different classes
print(classification_report(test_classes, vr_predict_classes, labels=labels))

In [ ]:
#Optional if you would like each of these metrics separately
#[precision,recall,fscore,support]=precision_recall_fscore_support(test_classes, vr_predict_classes, labels=labels)
#print "precision: ", precision
#print "recall: ", recall
#print "f1 score: ", fscore
#print "support: ", support